In [49]:
import pandas as pd
import numpy as np
#from pandas.io.common import is_url
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.offline as offline
#offline.init_notebook_mode()
import cufflinks as cf
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cf.go_offline()
cf.set_config_file(world_readable=True, theme='solar')



ModuleNotFoundError: No module named 'cufflinks'

In [40]:
path="D:/scrapping/magicbricks/"
df=pd.read_csv(f'{path}data_gnr_jan.csv')

In [41]:
df.description[0]

'2 Bath,Unfurnished,16 floor Paramount Golf Forest Ac Apartments complimented by designer landscape with a green golf course for you and your family, a well-deserved healthy lifestyle.'

In [13]:
print('We have ', len(df), 'properties in the data')

We have  1555 properties in the data


In [14]:
def print_description(society):
    example = df[df.Society == society][['description', 'name']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [15]:
print_description('Mahagun Mantra 2')

2 Bath,Semi-Furnished,16 floor,North - East facing Mahagun  A name that performs has come up with their worldclass residential project Mahagun Mantra 2. The project is spread over 9 acres of land and the site is filled with plants and trees in order to ensure a natural and healthy environment. The project has 4 towers along with an iconic tower villaments which are based on Indian Vastu design.Mahagun Mantra 2 Noida Extension has to offer 2 BHK apartments in sizes ranging from 850 sq. ft. to 1125 sq. ft. The apartments are designed by wellknown architects and designers Hafeez Contractor. The base of the entire building is made using RC
Name: 2 BHK Apartment for Sale in Mahagun Mantra 2, Greater Noida West


In [16]:
df.Society.value_counts()

Purvanchal Royal City     27
Gaur City                 23
Gaur Atulyam              22
Gaur Saundaryam           18
Paramount Golf Foreste    18
                          ..
Stellar Icon               1
Nirala Aspire              1
Defence Empire             1
Sikka Karnam Greens        1
Sea Court                  1
Name: Society, Length: 187, dtype: int64

In [17]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]



In [58]:
df['description']=df['description'].replace('is_url','')

In [18]:
common_words = get_top_n_words(df['description'], 20)
df1 = pd.DataFrame(common_words, columns = ['description' , 'count'])


In [19]:
if  df['description'].str.contains('link').any():
    print(df['description'])

0       2 Bath,Unfurnished,16 floor Paramount Golf For...
1       2 Bath,Semi-Furnished SKA Metro Ville is the f...
2       2 Bath,Semi-Furnished We at Prideville underst...
3       4 Bath,Semi-Furnished,19 floor A life where yo...
4       2 Bath,Unfurnished In Greater Noida(West), alo...
                              ...                        
1550    5 Bath,Semi-Furnished Luxury is not common. Lu...
1551    2 Bath,Semi-Furnished Gaur City is one of the ...
1552    5 Bath,Semi-Furnished A life where you can liv...
1553    4 Bath,Semi-Furnished RISE RESORT RESIDENCES i...
1554    5 Bath,Semi-Furnished Luxury is not common. Lu...
Name: description, Length: 1555, dtype: object


In [20]:
df['description']=df['description'].astype(str)

In [103]:
del df['url']

In [21]:
df

,Unnamed: 0,Society,addressLocality,agentCompanyName,agentMaskedmobilenumber,agentName,bathroom,bedroom,builderName,cityName,createdDate,description,floorSize,floorno,furnshingstatus,id,latitude,locality,longitude,name,numberOfRooms,postedBy,price,priceInWord,projectName,propertyType,url
0,0,Golfforeste AC Apartments,Zeta 1,BD Green Home Pvt. Ltd.,+91-88XXXXXXX90,Praveen Choudhary,2.0,2.0,Paramount Group,Greater Noida,20200128,"2 Bath,Unfurnished,16 floor Paramount Golf For...",1260 FTK,16,Unfurnished,domcache_srp_46630105,NaN,Zeta 1,NaN,2 BHK Apartment for Sale in Golfforeste AC Apa...,2.0,Agent,3710000.0,37.1 Lac,Golfforeste AC Apartments,Apartment,https://www.magicbricks.com/propertyDetails/2-...
1,1,SKA Metro Ville,"Eta 2, Eta",NaN,+91-91XXXXXXX15,SKA GROUP,2.0,2.0,SKA Group,Greater Noida,20200128,"2 Bath,Semi-Furnished SKA Metro Ville is the f...",920 FTK,NaN,Semi-Furnished,domcache_srp_46365021,77.537536,"Eta 2, Eta",28.496992,"2 BHK Apartment for Sale in SKA Metro Ville, E...",2.0,Builder,2560000.0,25.6 Lac,SKA Metro Ville,Apartment,https://www.magicbricks.com/propertyDetails/2-...
2,2,Prideville,Yamuna Expressway,Imperia Structures Ltd.,+91-74XXXXXXX93,Vipin,2.0,2.0,Imperia Structures Ltd.,Greater Noida,20200107,"2 Bath,Semi-Furnished We at Prideville underst...",900 FTK,NaN,Semi-Furnished,domcache_srp_45300283,NaN,Yamuna Expressway,NaN,"2 BHK Apartment for Sale in Prideville , Yamun...",2.0,Builder,4680000.0,46.8 Lac,Prideville,Apartment,https://www.magicbricks.com/propertyDetails/2-...
3,3,Purvanchal Royal City,Chi 5,Purvanchal Projects Pvt. Ltd.,+91-78XXXXXXX42,Shashi Kapoor,4.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,20200109,"4 Bath,Semi-Furnished,19 floor A life where yo...",2505 FTK,19,Semi-Furnished,domcache_srp_46527183,77.506133,Chi 5,28.430194,4 BHK Apartment for Sale in Purvanchal Royal C...,4.0,Builder,11200000.0,1.12 Cr,Purvanchal Royal City,Apartment,https://www.magicbricks.com/propertyDetails/4-...
4,4,Stellar One,Greater Noida West,Stellar Ventures Pvt. Ltd.,+91-98XXXXXXX77,Stellar Ventures Pvt. Ltd.,2.0,2.0,Stellar Ventures Pvt. Ltd.,Greater Noida,20191230,"2 Bath,Unfurnished In Greater Noida(West), alo...",895 FTK,NaN,Unfurnished,domcache_srp_42326915,77.436005,Greater Noida West,28.576390,"2 BHK Apartment for Sale in Stellar One, Great...",2.0,Builder,3110000.0,31.1 Lac,Stellar One,Apartment,https://www.magicbricks.com/propertyDetails/2-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,1556,Godrej Golf Links,Sector 27,Home Consultant Pvt. Ltd.,+91-99XXXXXXX66,Dhiraj Kumar,5.0,4.0,Godrej Properties,Greater Noida,20200114,"5 Bath,Semi-Furnished Luxury is not common. Lu...",2506 FTK,NaN,Semi-Furnished,domcache_srp_47254615,77.535673,Sector 27,28.470149,"4 BHK Villa for Sale in Godrej Golf Links, Sec...",4.0,Agent,15300000.0,1.53 Cr,Godrej Golf Links,Villa,https://www.magicbricks.com/propertyDetails/4-...
1551,1557,Gaur City,Greater Noida West,Property Plus Realtors,+91-99XXXXXXX48,Isha Pandey,2.0,3.0,Gaurs Group,Greater Noida,20191018,"2 Bath,Semi-Furnished Gaur City is one of the ...",110 FTK,NaN,Semi-Furnished,domcache_srp_45326637,77.434908,Greater Noida West,28.604830,"3 BHK Apartment for Sale in Gaur City, Greater...",3.0,Agent,3380000.0,33.8 Lac,Gaur City,Apartment,https://www.magicbricks.com/propertyDetails/3-...
1552,1558,Purvanchal Royal City,Chi 5,SMC Real Estate Advisors Pvt. Ltd.,+91-74XXXXXXX40,Rajat Chandra,5.0,4.0,Purvanchal Projects Pvt. Ltd.,Greater Noida,20200109,"5 Bath,Semi-Furnished A life where you can liv...",2505 FTK,NaN,Semi-Furnished,domcache_srp_47219937,77.506133,Chi 5,28.430194,4 BHK Apartment for Sale in Purvanchal Royal C...,4.0,Agent,11200000.0,1.12 Cr,Purvanchal Royal City,Apartment,https://www.magicbricks.com/propertyDetails/4-...
1553,1559,Rise Resort Residences,Greater Noida West,Wealth Clinic Pvt. Ltd.,+91-92XXXXXXX31,Sumit Raheja,4.0,4.0,Rise Group,Greater Noida,20200127,"4 Bath,Semi-Fur

In [33]:
df1.groupby('description').sum()['count'].sort_values().iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 words in property description before removing stop words')

AttributeError: 'Series' object has no attribute 'iplot'

In [69]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [70]:
common_words = get_top_n_trigram(df['description'], 20)
df5 = pd.DataFrame(common_words, columns = ['description' , 'count'])
df5.groupby('description').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')


AttributeError: module 'plotly.validators.carpet' has no attribute 'HoverinfoValidator'

In [71]:
df5

ImportError: cannot import name 'is_url' from 'pandas.io.common' (C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\common.py)

                    description  count
0      multistorey apartment is    829
1           bath semi furnished    712
2            available for sale    548
3              is available for    539
4                   for sale in    488
5        apartment is available    487
6             greater noida for    456
7     bhk multistorey apartment    407
8          semi furnished floor    375
9         floor bhk multistorey    347
10                is located at    333
11       bath unfurnished floor    326
12         apartment is located    308
13               in building of    246
14            building of total    245
15  floor multistorey apartment    198
16            floor east facing    197
17            north east facing    195
18               floor in tower    175
19                  in tower of    175

In [54]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3), stop_words='english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

In [73]:
common_words = get_top_n_trigram(df['description'], 20)
df6 = pd.DataFrame(common_words, columns = ['description' , 'count'])

In [74]:
df6

ImportError: cannot import name 'is_url' from 'pandas.io.common' (C:\ProgramData\Anaconda3\lib\site-packages\pandas\io\common.py)

                        description  count
0               bath semi furnished    712
1          apartment available sale    491
2   multistorey apartment available    448
3         bhk multistorey apartment    407
4              semi furnished floor    375
5             floor bhk multistorey    347
6            bath unfurnished floor    326
7       floor multistorey apartment    311
8     multistorey apartment located    295
9                 floor east facing    197
10                north east facing    195
11                floor tower total    175
12                 floor north east    159
13              furnished floor bhk    143
14               greater noida west    138
15             floor building total    130
16         apartment building total    115
17             bath furnished floor    109
18            available sale sector    104
19   multistorey apartment building    103

In [75]:
df['word_count'] = df['description'].apply(lambda x: len(str(x).split()))
desc_lengths = list(df['word_count'])
print("Number of descriptions:",len(desc_lengths),
      "\nAverage word count", np.average(desc_lengths),
      "\nMinimum word count", min(desc_lengths),
      "\nMaximum word count", max(desc_lengths))

Number of descriptions: 1555 
Average word count 37.82508038585209 
Minimum word count 4 
Maximum word count 469


In [76]:
df['word_count'].iplot(
    kind='hist',
    bins = 50,
    linecolor='black',
    xTitle='word count',
    yTitle='count',
    title='Word Count Distribution in Hotel Description')

AttributeError: module 'plotly.validators.carpet' has no attribute 'HoverinfoValidator'

In [50]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
url_symbol=re.compile('[^https?:\/\/.*[\r\n]*]')
STOPWORDS = set(stopwords.words('english'))

In [51]:
def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text =url_symbol.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text

In [52]:
df['desc_clean'] = df['description'].apply(clean_text)

In [55]:
common_words = get_top_n_trigram(df['desc_clean'], 20)
df6 = pd.DataFrame(common_words, columns = ['desc_clean' , 'count'])
df6.groupby('desc_clean').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description after removing stop words')


AttributeError: 'Series' object has no attribute 'iplot'

In [56]:
df6

,desc_clean,count
0,apartment available sale,490
1,multistorey apartment available,448
2,bhk multistorey apartment,407
3,bath semifurnished floor,374
4,floor bhk multistorey,347
5,bath unfurnished floor,326
6,floor multistorey apartment,311
7,multistorey apartment located,296
8,floor east facing,197
9,north east facing,190


In [67]:
df.set_index('name', inplace = True)
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(df['desc_clean'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

KeyError: "None of ['name'] are in the columns"

In [76]:
indices = pd.Series(df.index)

In [86]:
def recommendations(name, cosine_similarities = cosine_similarities):
    
    recommended_properties = []
    
    # gettin the index of the hotel that matches the name
    if indices[indices == name].shape[0]>0:
        idx = indices[indices == name].index[0]
    else:
        idx=clean_text(name)
        tfidf_matrix = tf.fit_transform(idx)
    print(idx)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_20_indexes = list(score_series.iloc[1:21].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_20_indexes:
        recommended_properties.append(list(df.index)[i])
        
    return recommended_properties

In [87]:
recommendations('2 BHK Apartment for Sale in Arihant Ambar, Greater Noida West')

ValueError: Iterable over raw text documents expected, string object received.

In [95]:
count = CountVectorizer()
count_vector = count.fit(df['description'])

In [97]:
count_matrix = count.fit_transform(df['description'])

In [96]:
print(count_vector.vocabulary_)

{'bath': 620, 'unfurnished': 3293, '16': 90, 'floor': 1448, 'paramount': 2399, 'golf': 1568, 'forest': 1463, 'ac': 341, 'apartments': 486, 'complimented': 926, 'by': 757, 'designer': 1094, 'landscape': 1942, 'with': 3467, 'green': 1593, 'course': 1001, 'for': 1461, 'you': 3501, 'and': 475, 'your': 3504, 'family': 1376, 'well': 3432, 'deserved': 1089, 'healthy': 1656, 'lifestyle': 1984, 'semi': 2831, 'furnished': 1507, 'ska': 2907, 'metro': 2144, 'ville': 3386, 'is': 1847, 'the': 3180, 'fresh': 1486, 'launch': 1954, 'residential': 2712, 'group': 1608, 'housing': 1720, 'project': 2566, 'in': 1768, 'greater': 1589, 'noida': 2275, 'igbc': 1740, 'gold': 1566, 'pre': 2524, 'certified': 807, 'which': 3445, 'aims': 422, 'to': 3223, 'promote': 2571, 'homes': 1698, 'where': 3442, 'residents': 2713, 'could': 994, 'live': 2001, 'quality': 2613, 'optimal': 2345, 'use': 3328, 'of': 2311, 'natural': 2230, 'resources': 2721, 'covers': 1010, 'an': 473, 'area': 517, '20073': 132, '90': 313, 'sq': 3010, 